Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Build ML Pipeline with if-else node

In the [get started](../../../samples/basics//get-started.ipynb) tutorial we have introduced how to build an ML pipeline by using Azure Machine Learning components. In this tutorial, you will learn how to build pipeline with if-else node.



## Prerequisites
* Install private version of packages following the [instructions here](../README.md).

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Enable private features

In [1]:
# Import required libraries
import os


# enable private features
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "True"

## 1.2 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91"
resource_group = "automlimage_eastus2_rg"
workspace = "risha-ws-eastus2"
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "gpu-cluster-nc6s"
print(ml_client.compute.get(cluster_name))

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'gpu-cluster-nc6s', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/automlimage_eastus2_rg/providers/Microsoft.MachineLearningServices/workspaces/risha-ws-eastus2/computes/gpu-cluster-nc6s', 'Resource__source_path': None, 'base_path': 'E:\\AutoML\\Repos\\azureml-examples\\sdk\\python\\jobs\\pipelines\\1j_pipeline_with_condition_flow\\if-else', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000195E2632D00>, 'resource_id': None, 'location': 'eastus2', 'size': 'STANDARD_NC6S_V3', 'min_instances': 1, 'max_instances': 22, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated', 'subnet': None})


In [3]:
# %pip install azure-ai-ml==1.1.0a20221019005 --extra-index-url https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/
%pip install azure-ai-ml
%pip install mldesigner==0.0.80281497 --extra-index-url=https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2

Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2
Note: you may need to restart the kernel to use updated packages.


## 2. Build pipeline with if-else node
### 2.1 Create a component with conditional output

In [4]:
from framework_selector_with_boolean_output import framework_selector

[2023-01-09 15:59:19,627][mldesigner][WARNING] - Mldesigner requires azure-ai-ml >= 1.2.0 package to be fully functional.It's highly recommended to install the latest azure-ai-ml package.


### 2.2 pipeline with if-else node

In [5]:
from mldesigner.dsl import condition # will merger to core sdk package
from azure.ai.ml import Input, load_component
from azure.ai.ml.dsl import pipeline


hf_component = load_component(
    source="dummy_hf_od_component.yaml"
)

#automl_image_od_component = load_component(
#    source="azureml://registries/azureml-staging/components/train_object_detection_model/versions/0.0.2-previewazureml-staging.1662484049"
#)

automl_image_od_component = load_component(
        client=ml_client, name="train_object_detection_model", version="3")

@pipeline()
def condition_pipeline():
    result = framework_selector()
    runtime = automl_image_od_component(
        training_data="azureml://datastores/workspaceblobstore/paths/vision-od/train/",
        validation_data="azureml://datastores/workspaceblobstore/paths/vision-od/valid/"
    )
    hf = hf_component(model_name='yolov5')
    condition(condition=result.outputs.output, false_block=hf, true_block=runtime)

pipeline_job = condition_pipeline()

# set pipeline level compute
pipeline_job.settings.default_compute = cluster_name

In [6]:
# validate pipeline
ml_client.jobs.validate(pipeline_job)

Method validate: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{
  "result": "Succeeded"
}

### 2.3 Run pipeline job

In [7]:
# pipeline parameter can be overridden when submit pipeline
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)

# show detail information of job
pipeline_job

Uploading if-else (0.03 MBs): 100%|##########################################################| 34823/34823 [00:00<00:00, 80897.09it/s]




Experiment,Name,Type,Status,Details Page
pipeline_samples,sincere_pump_b6rt83ss7f,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)